In [1]:
#In this notebook, we estimate the time constants T_d and T_p using linear regression by taking time estimations E[T] of many points with different N and K values for 1 dimensional case

In [107]:
import numpy as np
import random
import time
import math

In [108]:
def CollisionCalc(X0, X1, R):
    return abs(X0-X1) < (2 * R)

def TotalCollisions(Points, R):
    calc=  0
    for i in range(len(Points)):
        j = i+1
        while j < len(Points):
            calc+=CollisionCalc(Points[i], Points[j], R)
            j+=1

    return calc

In [109]:
def generateSample(N, K, Samples=1000):
    Dataset = []
    DatasetK = np.random.random((Samples, K))
    for x in range(Samples):
        Partitions = [[] for x in range(N)]
        #add values to partitions
        for i in range(K):
            Partitions[int((DatasetK[x][i]*N)-(1e-5))].append(DatasetK[x][i])

        Dataset.append(Partitions)
    return Dataset

In [125]:
def SampleTest(N, Dataset, R=0.004):
    Cols = []
    for dat in Dataset:
        Cols.append(0)
        for i in range(N-1):
            if len(dat[i]) == 0:
                continue
            Cols[-1]+=TotalCollisions(dat[i]+dat[i+1], R)
    return Cols

In [126]:
def EstimateTimeSample(N, K, R=0.004, Samples=1000):
    DataSet = generateSample(N, K, Samples=Samples)
    tstart = time.time()
    Ans = SampleTest(N, DataSet, R = R)
    return (time.time()-tstart)/Samples

In [112]:
def DataSet(Nlow, Nhigh, Nstep, Klow, Khigh, Kstep, Samples = 20):
    timeSteps = []
    for i in range(Samples):
        N = random.randint(int(Nlow/Nstep), int(Nhigh/Nstep))*Nstep
        K = random.randint(int(Klow/Kstep), int(Khigh/Kstep))*Kstep
        timeSteps.append([N, K, EstimateTimeSample(N, K, R=0.004, Samples=100)])
    return timeSteps

In [113]:
def DataSetPrep(Dataset):
    #change the (N, K, E[T]) fields in dataset to (N, 2K(K-1)(N-1)/N^2, E[T])
    Dat = []
    for x in range(len(Dataset)):
        n = Dataset[x][0]
        k = Dataset[x][1]
        t = Dataset[x][2]
        Dat.append([n, 2*k*(k-1)*(n-1)/(n*n), t])
    return Dat

In [114]:
X = DataSet(100, 3000, 100, 20, 300, 10, Samples=300)

In [115]:
X = np.array(X)
print(X[0])

[1.50000000e+03 2.00000000e+01 1.84402466e-04]


In [116]:
Y = DataSetPrep(X)

In [117]:
def SampleDatasetToLinearRegForm(Dataset):
    X = []
    Y = []
    for d in Dataset:
        X.append([d[0], d[1], 1])
        Y.append(d[2])
    return (np.array(X), np.array(Y))

In [118]:
Z = SampleDatasetToLinearRegForm(Y)

In [119]:
def linearRegression(X, Y):
    Vars = np.matmul(np.linalg.inv(np.matmul(X.transpose(), X)), np.matmul(X.transpose(), Y))
    return Vars

In [120]:
T_p, T_d, C = linearRegression(Z[0], Z[1])

In [121]:
print(T_p, T_d, C)
print(T_d/T_p)

8.817541768843517e-08 4.6558786390917547e-07 0.00010648845090977291
5.280245629845659


In [122]:
ratio = T_d/T_p
Value = np.sqrt(2 * X[:,1] * (X[:,1]-1) * (X[:,0]-1) * ratio)

In [123]:
print(Value-X[:,0])

[  952.64490527  4167.2082024   2029.59399401 14486.46212227
 10696.4273801   6525.70353583  9260.17244539 29832.92135247
 11003.63335554  5656.54093984 37078.11287872 18537.38331219
  5057.25032038 15005.70242616  5278.13464808  3943.46071914
 39005.47229567 29780.0099438  36535.34867859  9260.17244539
 20834.50675044 28785.15823283 22454.67399152 21825.71192496
  1177.09133195  4028.98461579 35346.30877845 21241.45250279
  5985.90839177 25481.02870627 39513.03553645 21725.39788281
 15362.512104   19017.97482448 12085.68798648  4927.50364069
 30821.71400288 31436.16142421 10176.89611573 14306.73864362
 24227.8733796  30981.00478332 30380.66199342 40755.1863365
 42504.90009524 26711.3354225   4616.36200849 19826.06311354
  4111.30379613 27604.44055938 42087.53083415   770.61571167
 36535.34867859 37078.11287872  8614.00249822   737.40948413
 27933.46895792 44254.61360328 23037.0224999   7779.24338552
  4729.1421696  16527.49151708  3990.56341226 34399.13306224
  7464.44936541 20518.282

In [124]:
#conclusion: In 1-dimensional case, the most optimized version of the grid is where the grid size is equal to the ball size radius 0.004
#This is true for most grid sizes unless the balls are very small in size